In [0]:
import os
from xml.dom import minidom
import pandas as pd
import numpy as np 
import nltk
import gzip
import gensim 
import logging,re
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Extracting text from Query and statue documents**

In [0]:
directory = os.getcwd()
dirr = '/content/drive/My Drive/AILA-data/'
tmp = open(dirr+"Query_doc.txt", "r") 
temp = tmp.read()
tmp.close()
queries = temp.split("\n")
queries.remove(queries[-1])
query_docs = []
query_labels = []
query_words = []
for i in range(len(queries)):
    elem = queries[i].split("||")
    query_docs.append(elem[1])
    query_labels.append(elem[0])
    query_words.append(re.split('\W+',elem[1]))

In [0]:
# Extract text from inside

def preprocess(directory):
    statute_files = os.listdir(directory)
    statute_docs =[]
    statute_words = []
    for i in range(len(statute_files)):
        if i%50 ==0:
            print("Processing : " , i )
        path = directory+'/'+statute_files[i]
        f = open(path, "r")
        statute_docs.append(f.read())
        statute_words.append(re.split('\W+',statute_docs[i]))
    return statute_docs,statute_words,statute_files

prior_statute_docs, prior_statute_words,prior_statute_labels = preprocess(dirr+'Object_statutes')
  
data =  query_docs + prior_statute_docs
words = query_words + prior_statute_words
labels = query_labels + prior_statute_labels

Processing :  0
Processing :  50
Processing :  100
Processing :  150


**Pre-processing the text**

In [0]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import re
from nltk.corpus import stopwords

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('words')
lemma = WordNetLemmatizer()
lemma_dictionary = {}
stop_words = set(stopwords.words('english'))
dict_words = set(nltk.corpus.words.words())
def filt(statute_docs):
    filtered_words =[]
    filtered_docs = [ '' for i in range(len(statute_docs))]
    lemma_docs = [ '' for i in range(len(statute_docs))]
    for i in range(len(statute_docs)):
        if i%50 ==0:
            print("Processing : ", i)
        statute_words = re.split("(?:(?:[^a-zA-Z]+')|(?:'[^a-zA-Z]+))|(?:[^a-zA-Z']+)", statute_docs[i])
        filtered_word_list = [word.lower() for word in statute_words if (( len(word) >= 3 and word.lower() not in stop_words ))  ]    #removing stop words
        filtered_words.append(filtered_word_list)
        for word in filtered_word_list:
            filtered_docs[i] = filtered_docs[i] + word + " "
            lemma_docs[i] = lemma_docs[i] + lemma.lemmatize(word) + " "  #word net lemmatization
    return lemma_docs
filtered_docs = filt(data)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
Processing :  0
Processing :  50
Processing :  100
Processing :  150
Processing :  200


**BM25 Method**

In [0]:
from gensim.summarization.bm25 import get_bm25_weights
from gensim.summarization.bm25 import BM25


sim = [[] for i in range(40)]

query_no = 0
batch_size = 200 # BM25 on batch of 50 at a time. Insufficient memory in case of all at once.
for t in range((len(data)-50)// batch_size +1): 
  print(t)
  arr = []
  docs_seperator = filtered_docs[:50] + filtered_docs[50+batch_size*t:50+batch_size*(t+1)]
  for elem in docs_seperator:
    arr.append(elem.split())
  corpus = arr
  result = get_bm25_weights(corpus, n_jobs=-1)
  
  for i in range(50,len(result)):
    for z in range(40):
      x = result[z][z]
      sim[z].append(result[z+10][i]/x)

0


**Saving the similarity scores of Query with case documents in decreasing order of required format for BM25 model**

In [0]:
f = open("BM25-task2.txt","w+")
for z in range(40):
  lst = []
  assert len(sim[z]) == 197
  for i in range(len(sim[z])):
    lst.append((sim[z][i],i))
  lst.sort(reverse = True)
  for i in range(len(lst)):
    f.write("AILA_Q"+str(z+11) +" "+ "Q0" + " "+ prior_statute_labels[lst[i][1]][:-4] + " "+  str(i+1) +" "+ str( lst[i][0])+ " " + "cat" + "\n")
    

**UPLOADING goldstd_statute.txt file **

In [0]:
from google.colab import files
uploaded = files.upload()

Saving goldstd_statute.txt to goldstd_statute.txt


**Another Method**

**TF-IDF Method**

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
tfidf = vect.fit_transform(filtered_docs)
tf_idf = (tfidf * tfidf.T).A

In [0]:
simm = [[] for i in range(40)]
for i in range(50,len(tf_idf)):
  for z in range(40):
    simm[z].append(tf_idf[z+10][i])

**Saving the similarity scores of Query with case documents in decreasing order of required format for TF-IDF model**

In [0]:
f = open("TFIDF-task2.txt","w+")
for z in range(40):
  lst = []
  assert len(simm[z]) == 197
  for i in range(len(simm[z])):
    lst.append((simm[z][i],i))
  lst.sort(reverse = True)
  for i in range(len(lst)):
    f.write("AILA_Q"+str(z+11) +" "+ "S0" + " "+ prior_statute_labels[lst[i][1]][:-4] + " "+  str(i+1) +" "+ str( lst[i][0])+ " " + "cat2" + "\n")
    

**TREC_EVALUATION FOR TF-IDF METHOD**

In [0]:
!wget https://trec.nist.gov/trec_eval/trec_eval_latest.tar.gz

--2020-05-03 13:05:10--  https://trec.nist.gov/trec_eval/trec_eval_latest.tar.gz
Resolving trec.nist.gov (trec.nist.gov)... 132.163.4.36, 2610:20:6b01:4::36
Connecting to trec.nist.gov (trec.nist.gov)|132.163.4.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 181743 (177K) [application/x-gzip]
Saving to: ‘trec_eval_latest.tar.gz’

trec_eval_latest.ta 100%[===================>] 177.48K  --.-KB/s    in 0.1s    

2020-05-03 13:05:11 (1.75 MB/s) - ‘trec_eval_latest.tar.gz’ saved [181743/181743]



In [0]:
!tar -xf trec_eval_latest.tar.gz
!rm -r trec_eval_latest.tar.gz

In [0]:
cd trec_eval-9.0.7/

/content/trec_eval-9.0.7


**SCORES FOR Prec@5,10 and MAP**

In [0]:
!make

gcc -g -I.  -Wall -DVERSIONID=\"9.0.7\"  -o trec_eval trec_eval.c formats.c meas_init.c meas_acc.c meas_avg.c meas_print_single.c meas_print_final.c get_qrels.c get_trec_results.c get_prefs.c get_qrels_prefs.c get_qrels_jg.c form_res_rels.c form_res_rels_jg.c form_prefs_counts.c utility_pool.c get_zscores.c convert_zscores.c measures.c  m_map.c m_P.c m_num_q.c m_num_ret.c m_num_rel.c m_num_rel_ret.c m_gm_map.c m_Rprec.c m_recip_rank.c m_bpref.c m_iprec_at_recall.c m_recall.c m_Rprec_mult.c m_utility.c m_11pt_avg.c m_ndcg.c m_ndcg_cut.c m_Rndcg.c m_ndcg_rel.c m_binG.c m_G.c m_rel_P.c m_success.c m_infap.c m_map_cut.c m_gm_bpref.c m_runid.c m_relstring.c m_set_P.c m_set_recall.c m_set_rel_P.c m_set_map.c m_set_F.c m_num_nonrel_judged_ret.c m_prefs_num_prefs_poss.c m_prefs_num_prefs_ful.c m_prefs_num_prefs_ful_ret.c m_prefs_simp.c m_prefs_pair.c m_prefs_avgjg.c m_prefs_avgjg_Rnonrel.c m_prefs_simp_ret.c m_prefs_pair_ret.c m_prefs_avgjg_ret.c m_prefs_avgjg_Rnonrel_ret.c m_prefs_simp_imp.c 

**For BM25 method**

In [0]:
! ./trec_eval -m map -m P.5,10 ../goldstd_statute.txt ../BM25-task2.txt

map                   	all	0.0963
P_5                   	all	0.0850
P_10                  	all	0.0650


**For TFIDF method**

In [0]:
! ./trec_eval -m map -m P.5,10 ../goldstd_statute.txt ../TF-IDF-task2.txt

map                   	all	0.1350
P_5                   	all	0.1050
P_10                  	all	0.0825
